In [5]:
import pandas as pd
import numpy as np
from src.core.models import MLPModel,CNNModel

from src.utils.utils import load_data_with_all_features
from sklearn.model_selection import train_test_split

# Deep Learning Modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,MaxPooling1D,Flatten

**Feature Engineering**

In [3]:
data_all_features = load_data_with_all_features()

data_model = data_all_features[['open', 'high', 'low', 'close', 'volume btc', 'volume usdt',
 'tradecount', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
 'volume_em', 'volume_sma_em', 'volume_vpt','volatility_bbli',
 'volatility_kcc', 'volatility_kch', 'volatility_kcl','trend_visual_ichimoku_a',
 'trend_visual_ichimoku_b', 'trend_aroon_up','trend_aroon_down', 'trend_aroon_ind', 
 'trend_psar_up_indicator','momentum_stoch',
 'momentum_stoch_signal','month', 'day', 'hour', 'y']]

data_model = data_model.interpolate(axis=1)

In [11]:
train_loc = int(data_model.shape[0]*0.70)
X_train = data_model.drop(columns='y').iloc[:train_loc]
y_train = data_model.iloc[:train_loc].y

X_test = data_model.drop(columns='y').iloc[train_loc:]
y_test =  data_model.iloc[train_loc:].y

In [17]:
num_features = len(X_train.columns)
num_classes = 1

**MLP Model Creation**

In [27]:
modelo_mlp = Sequential([
    Dense(256,activation='relu',input_shape=(num_features,)),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(num_classes,activation='sigmoid')
])
modelo_mlp.compile(optimizer='adam',
                  loss='binary_crossentropy', # Pérdida ideal para clasificación binaria
                  metrics=['accuracy'])
modelo_mlp.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 256)            │         7,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,457 (201.00 KB)

 Trainable params: 51,457 (201.00 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
mlp_classifier = MLPModel(model=modelo_mlp)

MLPModel inicializado.


In [29]:
params_entrenamiento = {'epochs': 10, 'batch_size': 32, 'learning_rate': 'adam'}
mlp_classifier.fit(
    X_train,
    y_train,
    epochs=params_entrenamiento['epochs'],
    batch_size=params_entrenamiento['batch_size'],
    params=params_entrenamiento,
    verbose=1
)

Iniciando entrenamiento del modelo...
MLflow Run ID: 64cf36f2902d4633b3371f33b9afe36d


2025/10/28 09:56:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 09:56:56 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Entrenamiento finalizado. Métricas finales: {'final_accuracy': 0.5119005441665649, 'final_loss': 0.6928849816322327, 'final_val_accuracy': 0.5021178126335144, 'final_val_loss': 0.6932191848754883}


2025/10/28 09:57:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [30]:
mlp_classifier.save('../models/mlp_model.keras')

Modelo guardado en: ../models/mlp_model.keras


**CNN Model Creation**

In [33]:
cnn_model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(num_features,num_classes)),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(num_classes, activation='sigmoid')
    ])
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

In [34]:
cnn_classifier = CNNModel(model=cnn_model)

CNNModel inicializado.


In [35]:
X_train_reshaped = X_train.to_numpy().reshape((X_train.shape[0], num_features, 1))

# 5. Entrenar el modelo
params_entrenamiento = {'epochs': 10, 'batch_size': 32, 'learning_rate': 'adam'}
cnn_classifier.fit(
    X_train_reshaped,
    y_train,
    epochs=params_entrenamiento['epochs'],
    batch_size=params_entrenamiento['batch_size'],
    params=params_entrenamiento,
    verbose=1
)

Iniciando entrenamiento del modelo CNN...
MLflow Run ID: b3434dad51574fa6b928775d5f0dbfe4
Epoch 1/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.5038 - loss: 5001967.0000 - val_accuracy: 0.4794 - val_loss: 1200406.2500
Epoch 2/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5046 - loss: 1057591.1250 - val_accuracy: 0.5026 - val_loss: 32.3374
Epoch 3/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5096 - loss: 1359.2599 - val_accuracy: 0.5020 - val_loss: 10.9581
Epoch 4/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5102 - loss: 40.9821 - val_accuracy: 0.5017 - val_loss: 10.9789
Epoch 5/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5113 - loss: 3.9136 - val_accuracy: 0.5021 - val_loss: 10.5874
Epoch 6/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5119 - loss: 5.9951 - val_accuracy: 0.5023 - val_loss: 10.3094
Epoch 7/10
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.5119 - loss: 0.6948 - val_accuracy:

2025/10/28 10:00:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 10:00:43 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


Entrenamiento finalizado. Métricas finales: {'final_accuracy': 0.5119005441665649, 'final_loss': 0.6928464770317078, 'final_val_accuracy': 0.5023194551467896, 'final_val_loss': 10.297389030456543}


2025/10/28 10:00:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [36]:
cnn_classifier.save('../models/cnn_model.keras')

Modelo CNN guardado en: ../models/cnn_model.keras
